In [1]:
import numpy as np
from ipywidgets import widgets, Layout
import os
from pint import UnitRegistry

In [2]:
from IPython.display import display
w = widgets.IntSlider()
display(w)

In [3]:
display(w)

In [4]:
w.value = 50

In [5]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))

In [6]:
w = widgets.FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

display(w)

In [11]:
ur = UnitRegistry()

def filename_widget(description):
    entry = widgets.Text(value='', placeholder='filename', description=description, disabled=False)
    validation = widgets.Valid(value=False, description='file exists')

    def observer(change):
        try:
            filename = change['new']['value']
            validation.value = os.path.exists(filename) and os.path.isfile(filename)
        except (KeyError, TypeError):
            pass
    
    entry.observe(observer)
    return widgets.Box([entry, validation]), entry

input_file_box, input_file_text = filename_widget('Input file: ')
control_file_box, control_file_text = filename_widget('Control sample: ')

longitude_kw = widgets.Text(value='lon', description='Longitude keyword: ')
lattitude_kw = widgets.Text(value='lat', description='Lattitude keyword: ')
value_kw = widgets.Text(value='', placeholder='value name', description='Value keyword: ')

load_button = widgets.HBox([widgets.Button(description='Load', button_style='info')], layout=Layout(justify_content='flex-end'))

input_box = widgets.VBox([input_file_box, control_file_box, longitude_kw, lattitude_kw, value_kw, load_button])

######################################################################

tab_contents = ['All', 'Monthly']
children = [widgets.Label(value=name) for name in tab_contents]
tab = widgets.Tab()

month = widgets.Dropdown(
    options=dict(zip(['January', 'February', 'March', 'April', 'May', 'June'
                      'July', 'August', 'September', 'October', 'November', 'December'],
                     range(12))),
    value=0,
    description='Month:',
    disabled=False,
)
children[1] = month

tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])

selection_box = widgets.HBox([tab])

######################################################################

def quantity_widget(description, dimension):
    entry = widgets.Text(value='', placeholder='quantity of {}'.format(dimension),
                         description=description, disabled=False)
    validation = widgets.Valid(value=False, description='correct units')

    def observer(change):
        try:
            s = change['new']['value']
            v = ur(s)
            validation.value = (v.dimensionality == dimension)
            
        except Exception:
            pass
    
    entry.observe(observer)
    return widgets.Box([entry, validation]), entry

sigma_l_box, sigma_l = quantity_widget("spatial smoothing radius (gaussian std-dev)", "[length]")
sigma_t_box, sigma_t = quantity_widget("time smoothing interval (gaussian std-dev)", "[time]")
sensity_box, sensitivity_ratio = quantity_widget("sensitivity ratio", "[length]/[time]")
filtering_box = widgets.VBox([sigma_l_box, sigma_t_box, sensity_box])

######################################################################

accordion = widgets.Accordion(children=[input_box, selection_box,
                                        filtering_box])
accordion.set_title(0, 'Input specification')
accordion.set_title(1, 'Selection')
accordion.set_title(2, 'Filtering')

accordion

In [75]:
month.value

2